In [45]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import os
import warnings

warnings.filterwarnings('ignore')

# CONFIGURATION & LOAD
print("Loading dataset...")
FILE_PATH = r'C:\Users\arnav\OneDrive\Desktop\customer-risk-agent\data\raw\Online_Retail .xlsx'
PROCESSED_PATH = r'C:\Users\arnav\OneDrive\Desktop\customer-risk-agent\data\processed\labeled_customers.csv'

# Handle relative paths for flexibility
if not os.path.exists(FILE_PATH):
    FILE_PATH = os.path.join(os.getcwd(), '..', 'data', 'raw', 'Online_Retail.xlsx')

try:
    df_raw = pd.read_excel(FILE_PATH)
except:
    df_raw = pd.read_csv(FILE_PATH, encoding='ISO-8859-1')

df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

# Pre-calculate returns to avoid data loss during filtering
returns_df = df_raw[df_raw['Quantity'] < 0].groupby('CustomerID')['Quantity'].count().rename('Return_Count')

# Filter for valid sales
df = df_raw[(df_raw['Quantity'] > 0) & (df_raw['UnitPrice'] > 0)].dropna(subset=['CustomerID'])

# SPLIT STRATEGY (90-Day Lookahead)
max_date = df['InvoiceDate'].max()
cutoff_date = max_date - pd.Timedelta(days=90)

df_past = df[df['InvoiceDate'] <= cutoff_date].copy()
df_future = df[df['InvoiceDate'] > cutoff_date].copy()

print(f"Training Window: Up to {cutoff_date.date()}")
print(f"Target Window:   After {cutoff_date.date()}")

# FEATURE ENGINEERING
snapshot_date = cutoff_date + pd.Timedelta(days=1)

# Aggregation: Recency, Frequency, Tenure, Monetary
rfm = df_past.groupby('CustomerID').agg({
    'InvoiceDate': [
        lambda x: (snapshot_date - x.max()).days, # Recency
        lambda x: (snapshot_date - x.min()).days  # Tenure
    ],
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'UnitPrice': lambda x: (x * df_past.loc[x.index, 'Quantity']).sum()
}).fillna(0)

rfm.columns = ['Recency', 'Tenure', 'Frequency', 'Total_Items', 'Monetary_Sum']

# Merge metadata
rfm = rfm.merge(returns_df, on='CustomerID', how='left')
rfm['Return_Count'] = rfm['Return_Count'].fillna(0)

# Velocity Metrics
rfm['Tenure_Capped'] = rfm['Tenure'].clip(lower=30) 
recent_start = cutoff_date - pd.Timedelta(days=90)
recent_stats = df_past[df_past['InvoiceDate'] > recent_start].groupby('CustomerID')['InvoiceNo'].nunique().rename('Freq_Recent')
rfm = rfm.merge(recent_stats, on='CustomerID', how='left').fillna(0)

rfm['Velocity_Recent'] = rfm['Freq_Recent'] / 3
rfm['Velocity_Life'] = rfm['Frequency'] / (rfm['Tenure_Capped'] / 30)
rfm['Velocity_Drift'] = rfm['Velocity_Recent'] - rfm['Velocity_Life']

# Normalized Spend (Avg Monthly)
rfm['Avg_Monthly_Spend'] = rfm['Monetary_Sum'] / (rfm['Tenure_Capped'] / 30)

# Cap AOV outliers at 99th percentile
rfm['AOV'] = rfm['Monetary_Sum'] / rfm['Frequency']
cap_aov = rfm['AOV'].quantile(0.99)
rfm['AOV'] = rfm['AOV'].clip(upper=cap_aov)

# Feature subset for model
features = ['Recency', 'Frequency', 'Avg_Monthly_Spend', 'Tenure', 'AOV', 'Velocity_Recent', 'Velocity_Drift', 'Return_Count']
rfm_clean = rfm[features].copy()


# CLUSTERING (K-Means)


# Log transform for skew handling
rfm_log = np.log1p(rfm_clean[['Recency', 'Frequency', 'Avg_Monthly_Spend']])
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm_clean['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Dynamic Labeling (Centroid Analysis)
summary = rfm_clean.groupby('Cluster').agg({'Recency': 'mean', 'Avg_Monthly_Spend': 'mean'}).reset_index()
vip_idx = summary.sort_values('Avg_Monthly_Spend', ascending=False).iloc[0]['Cluster']
risk_idx = summary.sort_values('Recency', ascending=False).iloc[0]['Cluster']

def label_cluster(c):
    if c == vip_idx: return 'High Value'
    elif c == risk_idx: return 'At-Risk'
    else: return 'Average'

rfm_clean['Segment_Label'] = rfm_clean['Cluster'].apply(label_cluster)
rfm_clean['Target_Label'] = rfm_clean['Segment_Label'].map({'At-Risk': 0, 'Average': 1, 'High Value': 2})

# Calculate Future Spend (Target)
future_spend = df_future.assign(Spend=df_future['Quantity'] * df_future['UnitPrice']) \
                        .groupby('CustomerID')['Spend'].sum() \
                        .rename('FutureSales_Label')

rfm_final = rfm_clean.merge(future_spend, on='CustomerID', how='left').fillna(0)

# EVALUATION & EXPORT

print("\n" + "="*50)
print("CLUSTERING METRICS")
print("="*50)

sil_score = silhouette_score(rfm_scaled, rfm_clean['Cluster'])
db_score = davies_bouldin_score(rfm_scaled, rfm_clean['Cluster'])
ch_score = calinski_harabasz_score(rfm_scaled, rfm_clean['Cluster'])

print(f"{'Metric':<25} | {'Value':<10}")
print("-" * 40)
print(f"{'Silhouette Coeff':<25} | {sil_score:.4f}")
print(f"{'Davies-Bouldin Index':<25} | {db_score:.4f}")
print(f"{'Calinski-Harabasz':<25} | {ch_score:.1f}")

print("\n" + "="*50)
print("SEGMENT PROFILES")
print("="*50)

profile = rfm_final.groupby('Segment_Label')[['Recency', 'Frequency', 'Avg_Monthly_Spend', 'Tenure', 'FutureSales_Label']].mean()
profile['Count'] = rfm_final['Segment_Label'].value_counts()
profile['Share (%)'] = (profile['Count'] / len(rfm_final)) * 100
profile = profile[['Count', 'Share (%)', 'Recency', 'Frequency', 'Avg_Monthly_Spend', 'FutureSales_Label']]

print(profile.round(2).to_string())

# Save artifacts
if not os.path.exists(os.path.dirname(PROCESSED_PATH)): os.makedirs(os.path.dirname(PROCESSED_PATH))
rfm_final.to_csv(PROCESSED_PATH)
print(f"\nSaved processed data to: {PROCESSED_PATH}")

Loading dataset...
Training Window: Up to 2011-09-10
Target Window:   After 2011-09-10

CLUSTERING METRICS
Metric                    | Value     
----------------------------------------
Silhouette Coeff          | 0.3539
Davies-Bouldin Index      | 1.0516
Calinski-Harabasz         | 2727.1

SEGMENT PROFILES
               Count  Share (%)  Recency  Frequency  Avg_Monthly_Spend  FutureSales_Label
Segment_Label                                                                            
At-Risk         1375      40.80   168.52       1.34              51.52             310.12
Average         1438      42.67    55.73       2.72             251.03             505.80
High Value       557      16.53    17.83      11.05             765.54            3048.69

Saved processed data to: C:\Users\arnav\OneDrive\Desktop\customer-risk-agent\data\processed\labeled_customers.csv
